# Artificial Intelligence Course - Fall 1402
## Computer Assignment #4 - Reinforcement Learning

# Table of Contents

- [Part 1: Value Iteration & Policy Iteration Algorithms](#1)
    - [َQuestion 1:](#1-0)
    - [َQuestion 2:](#1-1)
    - [َQuestion 3:](#1-12)
    - [َQuestion 4:](#1-2)
    - [َQuestion 5:](#1-3)
        - [Value Iteration](#1-3-1)
        - [Policy Iteration](#1-3-2)
    - [َQuestion 6:](#1-4)
        - [Value Iteration](#1-4-1)
        - [Policy Iteration](#1-4-2)
- [Part 2: Q-Learning Algorithm](#2)
    - [َQuestion 7:](#2-0)
    - [َQuestion 8:](#2-1)
    - [َQuestion 9:](#2-2)
    - [َQuestion 10:](#2-3)

In [1]:
!pip install gym==0.25.2
!pip install numpy

In [2]:
# import
import numpy as np
import gym
import time
import matplotlib.pyplot as plt

<a name='1'></a>
## Part 1: Value Iteration & Policy Iteration Algorithms

In [3]:
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=False)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
# get familiar with the environment
print("you can see the environment in each step by render command :")
env.reset()
env.render()

you can see the environment in each step by render command :


/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


In [5]:
# Total no. of states
env.observation_space.n

16

In [6]:
# Total no. of actions
env.action_space.n

4

<a name='1-0'></a>
### Question 1:

Value Iteration is a dynamic programming algorithm that we use to find the optimal policy and tackle reinforcement learning problems. It is an approach that wants to compute the optimal state values iteratively. Firstly, we initialize the values of all states arbitrarily and consider a small positive number for Θ>0 as our accuracy of estimation. Next, we update each state's value using the Bellman equation. Precisely, value iteration is obtained by turning the Bellman equation into an update rule. This value iteration update is identical to the policy evaluation update, except the fact that it requires the maximum over all actions. This equation is written below: [1]

$$v_{k+1}(s)=max_{a}\sum_{s^{'},r}p(s^{'},r|s,a)[r+\gamma v_{k}(s^{'})]$$

While we update iteratively, we check whether the maximum change in the values of states in the current iteration is below our threshold or not. After the convergence of our values, we can drive the optimal policy from the computed values, and for each state, we choose the action that maximizes our state value. Hence, we can write: [1]

$$\pi(s)=argmax_{a}\sum_{s^{'},r}p(s^{'},r|s,a)[r+\gamma v_{k}(s^{'})]$$

Thus, we iteratively improved our estimates for state values using dynamic programming and value iteration algorithm.

<a name='1-1'></a>
### Question 2:

In [7]:
class ValueIteration():
    def __init__(self, env, discount_factor, theta=1e-8):
        self.env = env
        self.discount_factor = discount_factor
        self.theta = theta
        self.reset()
        self.state_values = np.ones((self.env.observation_space.n)) / self.env.action_space.n
        self.q_values = np.ones((self.env.observation_space.n, self.env.action_space.n)) / self.env.action_space.n
        self.state_values[self.env.observation_space.n - 1] = 0
        self.q_values[self.env.observation_space.n - 1] = np.zeros((self.env.action_space.n))

    def value_estimation(self):
        self.delta = np.inf

        while(self.delta > self.theta):

            self.delta = 0

            for state in range(self.env.observation_space.n):

                v = self.state_values[state]

                for action in range(self.env.action_space.n):
                    action_value = 0
                    for probability, next_state, reward, done in self.env.P[state][action]:
                         action_value += probability*(reward+self.discount_factor*self.state_values[next_state])
                    self.q_values[state, action] = action_value

                self.state_values[state] = np.max(self.q_values[state,:])

                self.delta = np.max([self.delta, abs(v - self.state_values[state])])

    def take_action(self, action):
        next_state, reward, done, _ = self.env.step(action)
        return next_state, reward, done

    def get_optimal_policy(self, state):
        return np.argmax(self.q_values[state,:])

    def get_state_values(self):
        return self.state_values

    def get_q_values(self):
        return self.q_values

    def reset(self):
        initial_state = self.env.reset()
        return initial_state

<a name='1-12'></a>
### Question 3:

Policy Iteration is a dynamic programming algorithm used for finding the optimal policy in reinforcement learning models. It consists of two main steps, called policy evaluation and policy improvement. Firstly, we initialize an arbitrarily policy and state value. Next, we use the iterative policy evaluation by updating each state's values using the Bellman equation under the current policy. Its equation is this: [1]

$$v(s)=\sum_{s^{'},r}p(s^{'},r|s,π(s))[r+\gamma v(s^{'})]$$

We repeat this equation until the change in state values is lower than our small threshold. For each state, we update the policy to be greedy with respect to our current state values, and this current policy is optimal if it does not change. Its representation is as below:

$$π(s)=argmax_{a}\sum_{s^{'},r}p(s^{'},r|s,a)[r+\gamma v(s^{'})]$$

We check if the policy has not changed, and if it changes, we repeat these steps.

<a name='1-2'></a>
### Question 4:

In [8]:
class PolicyIteration():
    def __init__(self, env, discount_factor, theta=1e-8):
        self.env = env
        self.discount_factor = discount_factor
        self.theta = theta
        self.reset()
        self.state_values = np.ones((self.env.observation_space.n)) / self.env.action_space.n
        self.q_values = np.ones((self.env.observation_space.n, self.env.action_space.n)) / self.env.action_space.n
        self.state_values[self.env.observation_space.n - 1] = 0
        self.q_values[self.env.observation_space.n - 1] = np.zeros((self.env.action_space.n))
        self.policy = np.random.randint(self.env.action_space.n, size=self.env.observation_space.n) # initial policy
        self.policy_stable = False

    def policy_evaluation(self):
        self.delta = np.inf

        while(self.delta >= self.theta):

            self.delta = 0

            for state in range(self.env.observation_space.n):

                v = self.state_values[state]

                new_state_value = 0
                for probability, next_state, reward, done in self.env.P[state][self.policy[state]]:
                    new_state_value += probability*(reward + self.discount_factor*self.state_values[next_state])
                self.state_values[state] = new_state_value

                self.delta = np.max([self.delta, abs(v - self.state_values[state])])

    def policy_improvement(self):
        self.policy_stable = True

        for state in range(self.env.observation_space.n):
            old_policy = self.policy[state]

            for action in range(self.env.action_space.n):

                action_value = 0
                for probability, next_state, reward, done in self.env.P[state][action]:
                    action_value += probability*(reward+self.discount_factor*self.state_values[next_state])
                self.q_values[state, action] = action_value

            self.policy[state] = np.argmax(self.q_values[state,:])

            if old_policy != self.policy[state]:
                self.policy_stable = False

    def policy_estimation(self):
        self.policy_stable = False

        while not self.policy_stable:
            self.policy_evaluation()
            self.policy_improvement()

    def take_action(self, action):
        next_state, reward, done, _ = self.env.step(action)
        return next_state, reward, done

    def get_optimal_policy(self, state):
        return self.policy[state]

    def get_state_values(self):
        return self.state_values

    def get_q_values(self):
        return self.q_values

    def reset(self):
        initial_state = self.env.reset()
        return initial_state

<a name='1-3'></a>
### Question 5:

<a name='1-3-1'></a>
#### Value Iteration:

In [9]:
optimal_policy_valueiteration = []
value_iteration = ValueIteration(env, discount_factor=0.9)
for i in range(100):
    value_iteration.value_estimation()
state_values = value_iteration.get_state_values()
q_values = value_iteration.get_q_values()
for state in range(env.observation_space.n):
    optimal_policy_valueiteration.append(value_iteration.get_optimal_policy(state))

In [10]:
state_values_table_valueiteration = np.array(state_values)
print('Here is our state value table for value iteration:')
print(state_values_table_valueiteration)

Here is our state value table for value iteration:
[5.90490000e-01 6.56100000e-01 7.29000000e-01 6.56100000e-01
 6.56100000e-01 2.60700746e-12 8.10000000e-01 2.60700746e-12
 7.29000000e-01 8.10000000e-01 9.00000000e-01 2.60700746e-12
 2.60700746e-12 9.00000000e-01 1.00000000e+00 0.00000000e+00]


In [11]:
q_values_table_valueiteration = np.array(q_values)
print('Here is our state-action value table for value iteration:')
print(q_values_table_valueiteration)

Here is our state-action value table for value iteration:
[[5.31441000e-01 5.90490000e-01 5.90490000e-01 5.31441000e-01]
 [5.31441000e-01 2.60700746e-12 6.56100000e-01 5.90490000e-01]
 [5.90490000e-01 7.29000000e-01 5.90490000e-01 6.56100000e-01]
 [6.56100000e-01 2.60700746e-12 5.90490000e-01 5.90490000e-01]
 [5.90490000e-01 6.56100000e-01 2.60700746e-12 5.31441000e-01]
 [2.60700746e-12 2.60700746e-12 2.60700746e-12 2.60700746e-12]
 [2.34630672e-12 8.10000000e-01 2.60700746e-12 6.56100000e-01]
 [2.60700746e-12 2.60700746e-12 2.60700746e-12 2.60700746e-12]
 [6.56100000e-01 2.60700746e-12 7.29000000e-01 5.90490000e-01]
 [6.56100000e-01 8.10000000e-01 8.10000000e-01 2.34630672e-12]
 [7.29000000e-01 9.00000000e-01 2.60700746e-12 7.29000000e-01]
 [2.60700746e-12 2.60700746e-12 2.60700746e-12 2.60700746e-12]
 [2.60700746e-12 2.60700746e-12 2.60700746e-12 2.60700746e-12]
 [2.34630672e-12 8.10000000e-01 9.00000000e-01 7.29000000e-01]
 [8.10000000e-01 9.00000000e-01 1.00000000e+00 8.10000000e-0

In [12]:
optimal_policy_table_valueiteration = np.array(optimal_policy_valueiteration).reshape((4, 4))
print('Here is our optimal policy table for value iteration:')
print(optimal_policy_table_valueiteration)

Here is our optimal policy table for value iteration:
[[1 2 1 0]
 [1 0 1 0]
 [2 1 1 0]
 [0 2 2 0]]


In [13]:
print('Here is our optimal policy graphical render for agent by using value iteration:')
first_state = env.reset()
env.render()
for i in range(15):
  time.sleep(2)
  action = optimal_policy_valueiteration[first_state]
  first_state = env.step(action)[0]
  env.render()

Here is our optimal policy graphical render for agent by using value iteration:


<a name='1-3-2'></a>
#### Policy Iteration:

In [14]:
optimal_policy_policyiteration = []
policy_iteration = PolicyIteration(env, discount_factor=0.9)
for i in range(100):
    policy_iteration.policy_estimation()
state_values = policy_iteration.get_state_values()
q_values = policy_iteration.get_q_values()
for state in range(env.observation_space.n):
    optimal_policy_policyiteration.append(policy_iteration.get_optimal_policy(state))

In [15]:
state_values_table_policyiteration = np.array(state_values)
print('Here is our state value table for policy iteration:')
print(state_values_table_policyiteration)

Here is our state value table for policy iteration:
[5.90490000e-01 6.56100000e-01 7.29000000e-01 6.56100000e-01
 6.56100000e-01 5.96399686e-13 8.10000000e-01 5.96399686e-13
 7.29000000e-01 8.10000000e-01 9.00000000e-01 5.96399686e-13
 5.96399686e-13 9.00000000e-01 1.00000000e+00 0.00000000e+00]


In [16]:
q_values_table_policyiteration = np.array(q_values)
print('Here is our state-action value table for policy iteration:')
print(q_values_table_policyiteration)

Here is our state-action value table for policy iteration:
[[5.31441000e-01 5.90490000e-01 5.90490000e-01 5.31441000e-01]
 [5.31441000e-01 5.36759718e-13 6.56100000e-01 5.90490000e-01]
 [5.90490000e-01 7.29000000e-01 5.90490000e-01 6.56100000e-01]
 [6.56100000e-01 5.36759718e-13 5.90490000e-01 5.90490000e-01]
 [5.90490000e-01 6.56100000e-01 5.36759718e-13 5.31441000e-01]
 [5.36759718e-13 5.36759718e-13 5.36759718e-13 5.36759718e-13]
 [5.36759718e-13 8.10000000e-01 5.36759718e-13 6.56100000e-01]
 [5.36759718e-13 5.36759718e-13 5.36759718e-13 5.36759718e-13]
 [6.56100000e-01 5.36759718e-13 7.29000000e-01 5.90490000e-01]
 [6.56100000e-01 8.10000000e-01 8.10000000e-01 5.36759718e-13]
 [7.29000000e-01 9.00000000e-01 5.36759718e-13 7.29000000e-01]
 [5.36759718e-13 5.36759718e-13 5.36759718e-13 5.36759718e-13]
 [5.36759718e-13 5.36759718e-13 5.36759718e-13 5.36759718e-13]
 [5.36759718e-13 8.10000000e-01 9.00000000e-01 7.29000000e-01]
 [8.10000000e-01 9.00000000e-01 1.00000000e+00 8.10000000e-

In [17]:
optimal_policy_table_policyiteration = np.array(optimal_policy_policyiteration).reshape((4, 4))
print('Here is our optimal policy table for policy iteration:')
print(optimal_policy_table_policyiteration)

Here is our optimal policy table for policy iteration:
[[1 2 1 0]
 [1 0 1 0]
 [2 1 1 0]
 [0 2 2 0]]


In [18]:
print('Here is our optimal policy graphical render for agent by using policy iteration:')
first_state = env.reset()
env.render()
for i in range(15):
  time.sleep(2)
  action = optimal_policy_policyiteration[first_state]
  first_state = env.step(action)[0]
  env.render()

Here is our optimal policy graphical render for agent by using policy iteration:


<a name='1-4'></a>
### Question 6:

Both the value and policy iteration methods returned the exact same policy. In terms of state values, there are slight differences in some states. Value iteration method returned slightly higher values for some of the states compared to policy iteration. However, this difference did not change our policy. By using the below code cells, we can see that the policy iteration method is much faster than value iteration.


<a name='1-4-1'></a>
#### Value Iteration:

In [19]:
time_elapsed_value_iteration = 0
for i in range(100):
    value_iteration = ValueIteration(env, discount_factor=0.9)
    time_1 = time.perf_counter() #Return the value of a performance counter
    value_iteration.value_estimation()
    time_2 = time.perf_counter()
    time_elapsed_value_iteration = time_elapsed_value_iteration + time_2 - time_1
print('Value iteration elapsed time for hundred episodes is: '+str(time_elapsed_value_iteration))

Value iteration elapsed time for hundred episodes is: 4.895099124000012


<a name='1-4-2'></a>
#### Policy Iteration:

In [20]:
time_elapsed_policy_iteration = 0
for i in range(100):
    policy_iteration = PolicyIteration(env, discount_factor=0.9)
    time_1 = time.perf_counter() #Return the value of a performance counter
    policy_iteration.policy_estimation()
    time_2 = time.perf_counter()
    time_elapsed_policy_iteration = time_elapsed_policy_iteration + time_2 - time_1
print('Value iteration elapsed time for hundred episodes is: '+str(time_elapsed_policy_iteration))

Value iteration elapsed time for hundred episodes is: 2.19284545399978


<a name='2'></a>
## Part 2: Q-Learning Algorithm

<a name='2-0'></a>
### Question 7:

Q-learning is an off-policy algorithm, and this means that this algorithm learns from the experiences of a policy that is different from our target policy. This policy can be driven by an exploration strategy, in which it explores the environment to discover our optimal actions. In this algorithm, we want to learn the state-action values, and they are calculated from this equation using the Bellman equation to update Q-values: [1]

$$Q(S_{t},A_{t})= Q(S_{t},A_{t}) + α[R_{t+1} + γmax_aQ(S_{t+1},a) - Q(S_{t},A_{t})]$$

Q-learning uses exploration and exploitation. With probability ϵ our algorithm selects a random action, and with probability 1-ϵ it selects the action with the highest Q-value. Hence, by using this algorithm, we can find the optimal action-value function independent of following a policy.

In [21]:
# hyperparameters
REPS = 20
EPISODES = 2000
EPSILON = 0.1
LEARNING_RATE = 0.1
DISCOUNT = 0.9
STUDENT_NUM = 280

In [22]:
# environment
env = gym.make('Taxi-v3')
env.seed(seed = 280)
Initial_State = env.reset()
Initial_State

/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


494

In [23]:
taxi_row, taxi_col, pass_idx, dest_idx = env.decode(Initial_State)
taxi_row, taxi_col, pass_idx, dest_idx

(4, 4, 3, 2)

In [24]:
# get familiar with the environment
print("you can see the environment in each step by render command :")
env.render()

you can see the environment in each step by render command :


In [25]:
# Total no. of states
env.observation_space.n

500

In [26]:
# Total no. of actions
env.action_space.n

6

<a name='2-1'></a>
### Question 8:

In our first episode, the value of Epsilon is one, and in further episodes, our value of Epsilon will decreases exponentially.

We used this method for our learning rate, except for the fact that our learning rate at episode zero is 0.99.

In [27]:
class QLearningAgent():
    def __init__(self, env, epsilon, learning_rate, discount_factor, seed):
      self.env = env
      self.epsilon = epsilon
      self.learning_rate = learning_rate
      self.olr = learning_rate
      self.discount_factor = discount_factor
      self.q_table = np.zeros((env.observation_space.n, env.action_space.n))
      self.seed = seed

    def choose_action(self, state):
      if np.random.rand() < self.epsilon:
        action = np.random.choice(self.env.action_space.n)
      else:
        action = self.get_optimal_policy(state)
      return action

    def update_q_table(self, state, action, nextState, reward):
      self.q_table[state][action] = self.q_table[state][action] + self.learning_rate*(reward+self.discount_factor*np.max(self.q_table[int(nextState),:])-(self.q_table[state][action]))

    def decay_epsilon(self, episode):
       self.epsilon = np.exp(-0.002*(episode+1))

    def decrease_learning_rate(self, episode):
      self.learning_rate = 0.99*np.exp(-0.002*(episode))

    def take_action(self, action):
      next_state, reward, done, _ = self.env.step(action)
      return next_state, reward, done

    def get_optimal_policy(self, state):
      return np.argmax(self.q_table[state])

    def get_q_values(self):
      return self.q_table

    def reset(self):
      return self.env.reset(seed=self.seed)

<a name='2-2'></a>
### Question 9:

In [28]:
agent = QLearningAgent(env, epsilon=0.1, learning_rate=0.1, discount_factor=0.9, seed=280)

optimal_state_values = []
optimal_q_values = []
optimal_policies = []

for repeat in range(REPS):
    env.seed(seed = 280)
    agent.reset()

    for episode in range(EPISODES):
        env.seed(seed = 280)
        state = agent.reset()
        total_reward = 0

        while True:
            action = agent.choose_action(state)
            next_state, reward, done = agent.take_action(action)
            agent.update_q_table(state, action, next_state, reward)
            total_reward += reward
            state = next_state

            if done:
                break

        agent.decay_epsilon(episode)
        agent.decrease_learning_rate(episode)

    optimal_state_values.append(np.max(agent.q_table, axis=1))
    optimal_q_values.append(agent.get_q_values())
for state in range(env.observation_space.n):
    optimal_policies.append(np.argmax(optimal_q_values[19][state]))

In [29]:
print('Here is our state values:')
print(optimal_state_values[19])

Here is our state values:
[ 0.          0.          7.7147      0.          0.          0.
 -4.99684549  0.          0.          0.          0.          0.
  0.          0.         -3.82326604  0.          0.          0.
  9.683       0.          0.          0.          5.94323     0.
  0.          0.         -4.44093943  0.          0.          0.
  0.          0.          0.          0.         -3.13696226  0.
  0.          0.          7.7147      0.          0.          0.
 -0.58568212  0.          0.          0.         -0.58568212  0.
  0.          0.          0.          0.          0.          0.
 -2.37440252  0.          0.          0.          5.94323     0.
  0.          0.         -1.52711391  0.          0.          0.
  0.4603532   0.          0.          0.          0.          0.
  0.          0.         -1.52711391  0.          0.          0.
  4.348907    0.          0.          0.         -2.3744027   0.
  0.          0.          1.62261467  0.          0.          0.

In [30]:
print('Here is our state-action values:')
print(optimal_q_values[19])

Here is our state-action values:
[[ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 4.348907    5.94323     4.348907    5.94323     7.7147     -3.05677   ]
 ...
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 1.62261467  2.9140163   1.62261467  2.9140163  -7.37738533 -7.37738533]
 [ 0.          0.          0.          0.          0.          0.        ]]


In [31]:
print('Here is our optimal policy:')
print(optimal_policies)

Here is our optimal policy:
[0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,

<a name='2-3'></a>
### Question 10:

In [32]:
print('Here is our optimal policy graphical render for our agent:')
env.seed(seed = 280)
first_state = env.reset()
env.render()
for i in range(15):
  action = optimal_policies[first_state]
  first_state = env.step(action)[0]
  time.sleep(3)
  env.render()

Here is our optimal policy graphical render for our agent:


# Reference



1. Sutton, Richard S.; Barto, Andrew G. Reinforcement Learning, second edition: An Introduction. MIT Press. ISBN 978-0-262-03924-6.